In [1]:
from img_doc.dataset import PubLayNetDataset
from img_doc.document import Document

from img_doc.document import Document, Block
from img_doc.document.page.extractors.block_extractors import PageAndWordClassificator, BaseRandomWalkClassificator

import numpy as np
import json
import random

import tensorflow as tf
import keras
from keras import layers

tf.random.set_seed(22)

import pandas as pd
from time import time
import matplotlib.pyplot as plt
import os

2024-03-01 21:01:47.532773: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-01 21:01:48.110035: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-01 21:01:48.110234: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-01 21:01:48.221475: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-01 21:01:48.504685: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-01 21:01:48.508996: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
pln_ds = PubLayNetDataset(r"/home/daniil/program/dataset/publaynet/", 
                          r"/home/daniil/program/dataset/tmp_publaynet/")

In [3]:
def shuffle(x_arrays, y_arrays):
    x_vec = np.concatenate(x_arrays)
    y_vec = np.concatenate(y_arrays)
    data = [(x, y) for x, y in zip(x_vec, y_vec)]
    random.shuffle(data)
    x_vec_rnd = np.array([x for x, y in data])
    y_vec_rnd = np.array([y for x, y in data])
    return x_vec_rnd, y_vec_rnd
    

def get_data(dataset, balans=True): #, batch_size=128

    X = np.array(dataset["x"])
    y = np.array(dataset["y"])
    
    keys = np.unique(y)
    Y_keys = dict()
    for i, key in enumerate(keys):
        z = np.zeros(5)
        z[i] = 1
        Y_keys[key] = z
    Y = np.array([Y_keys[yi] for yi in y])

    X_clus_list = []
    Y_clus_list = []

    for i, item in enumerate(Y_keys.keys()):
        X_clus_list.append(X[Y[:,i]==1,:])
        Y_clus_list.append(Y[Y[:,i]==1,:])
        
    if balans:
        balans_len = [len(yc) for yc in Y_clus_list]
        min_len = min(balans_len)
    
    train_x = []
    train_y = []
    test_x = []
    test_y = []
    val_x = []
    val_y = []
    for class_data in zip(X_clus_list, Y_clus_list):
        data = [(x, y) for x, y in zip(class_data[0], class_data[1])]
        random.shuffle(data)
        if balans:
            data = data[:min_len]
        len_data = len(data)
        len_test = round(0.2*len_data)
        len_val = round((len_data-len_test)*0.2)
        len_train = len_data - len_test - len_val
        
        train_x.append([x for x, y in data[:len_train]])
        train_y.append([y for x, y in data[:len_train]])
        
        val_x.append([x for x, y in data[len_train:len_train+len_val]])
        val_y.append([y for x, y in data[len_train:len_train+len_val]])
    
        test_x.append([x for x, y in data[-len_test:]])
        test_y.append([y for x, y in data[-len_test:]])
        
    train_x, train_y = shuffle(train_x, train_y)
    val_x, val_y = shuffle(val_x, val_y)
    test_x, test_y = shuffle(test_x, test_y)
    
    return train_x, train_y, val_x, val_y, test_x, test_y, Y_keys

def get_dataset(properties_brwc, properties_pwc, count_step, name):
    def fun_from_tmp_and_path_image(tmp_json, img_path):
        doc = Document()
        doc.set_from_path(img_path)
        doc.pages[0].set_blocks_from_dict(tmp_json["blocks"])
        doc.pages[0].set_words_from_dict(tmp_json["additional_info"]["words"])
        if "bold" in properties_brwc:
            doc.pages[0].extract_word_bold()
        for word in doc.pages[0].words:
            for block in doc.pages[0].blocks:
                if block.segment.is_intersection(word.segment):
                    block.words.append(word)
        x = []
        y = []
        for page in doc.pages:
            try:
                x =pwc.get_block_and_page_vecs(page)
                for block in page.blocks:
                    y.append(block.label)
            except:
                print(doc.path)
        return {"x": x, "y": y}
    brwc = BaseRandomWalkClassificator(conf={"properties":properties_brwc,"count_step":count_step})
    pwc = PageAndWordClassificator(brwc, conf={"properties":properties_pwc})
    rez = pln_ds.get_json_from_tmps_and_images(fun_from_tmp_and_path_image, balans = True)
    
    dataset = {"x": [], "y": []}
    for train_i in rez["train"]:
        for x in train_i["x"]:
            dataset["x"].append(x)
        for y in train_i["y"]:
            dataset["y"].append(y)

    train_x, train_y, val_x, val_y, test_x, test_y, Y_keys = get_data(dataset, balans=True)

    with open(name, "w") as f:
        json.dump({"train_x": train_x.tolist(), "train_y": train_y.tolist(),
                   "val_x": val_x.tolist(), "val_y": val_y.tolist(), 
                   "test_x": test_x.tolist(), "test_y": test_y.tolist()}, f)

In [ ]:
get_dataset(properties_brwc = ["many_dist", "many_angle", "place_in_block", "height", "bold"], 
            properties_pwc = ["place_in_page", "count_word_in_page"], count_step=100, name="dataset_1.json")

In [ ]:
name_model = "RWMDMAPHB-PLN"
with open("dataset_1.json", "r") as f:
    dataset = json.load(f)

In [ ]:
print("Первый x:\n", dataset["train_x"][0])
print("Последний, x:\n", dataset["train_x"][-1])
x_len = len(dataset["train_x"][0])
print("Размер x:\t", x_len)
print("Первый y:\t", dataset["train_y"][0])
print("Последний y:\t", dataset["train_y"][0])
print("Кол-во записей:\t", len(dataset["train_x"]))

In [ ]:
args = [[20, 50], [50, 50], [100, 50], [200, 50], [400, 50]]
mega_hist
for arg in args
    hist = []
    model = keras.Sequential()
    
    input_len = x_len
    output_len = 5
    
    model.add(layers.Dense(units=input_len, activation='relu'))
    # model.add(layers.Dense(units=100,  activation='relu'))
    # model.add(layers.Dropout(rate=0.2, seed=22))
    # model.add(layers.Dense(units=50,  activation='relu'))
    model.add(layers.Dropout(rate=0.2, seed=22))
    model.add(layers.Dense(units=25,  activation='relu'))
    model.add(layers.Dropout(rate=0.2, seed=22))
    model.add(layers.Dense(units=output_len, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    batch_size = 100
    epochs = 50
    history = model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, validation_data=(val_x, val_y))
    hist.append(history)

    loss_and_metrics = model.evaluate(test_x, test_y, batch_size=batch_size)
    loss_and_metrics
    mega_hist.append({"hist": hist, "loss_and_metrics": loss_and_metrics})

In [ ]:
with open(name, "w") as f:
        json.dump({"args": args,
                   "mega_hist":mega_hist, 
                   "test_x": test_x.tolist(), "test_y": test_y.tolist()}, f)